In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
import random
from __future__ import print_function
import pymysql

In [ ]:
html = "<head></head><body><h1>hello</h1></body>"

In [ ]:
soup = BeautifulSoup(html)

In [ ]:
print soup.prettify()

In [ ]:
webCrawl = webdriver.Chrome("/Users/stories2/Python/chromedriver")

In [ ]:
def ConnectToTargetUrl(url):
    webCrawl.get(url)

In [ ]:
def GetPitcherTeamTag(webCrawlDriver):
    return webCrawlDriver.find_element_by_id("cphContainer_cphContents_ddlPitcherTeam")

In [ ]:
def GetPitcherTeamList(webCrawlDriver):
    pitcherTeam = GetPitcherTeamTag(webCrawlDriver)
    return [data for data in pitcherTeam.find_elements_by_tag_name("option")]

In [ ]:
def GetPitcherNameTag(webCrawlDriver):
    return webCrawlDriver.find_element_by_id("cphContainer_cphContents_ddlPitcherPlayer")

In [ ]:
def GetPitcherNameList(webCrawlDriver):
    pitcherName = GetPitcherNameTag(webCrawlDriver)
    return [data for data in pitcherName.find_elements_by_tag_name("option")]

In [ ]:
def GetHitterTeamTag(webCrawlDriver):
    return webCrawlDriver.find_element_by_id("cphContainer_cphContents_ddlHitterTeam")

In [ ]:
def GetHitterTeamList(webCrawlDriver):
    hitterTeam = GetHitterTeamTag(webCrawlDriver)
    return [data for data in hitterTeam.find_elements_by_tag_name("option")]

In [ ]:
def GetHitterNameTag(webCrawlDriver):
    return webCrawlDriver.find_element_by_id("cphContainer_cphContents_ddlHitterPlayer")

In [ ]:
def GetHitterNameList(webCrawlDriver):
    hitterName = GetHitterNameTag(webCrawlDriver)
    return [data for data in hitterName.find_elements_by_tag_name("option")]

In [ ]:
def GetSearchButton(webCrawlDriver):
    return webCrawlDriver.find_element_by_id("cphContainer_cphContents_btnSearch")

In [ ]:
def GetBaseBallDataTable(webCrawlDriver):
    recordData = webCrawlDriver.find_element_by_id("cphContainer_cphContents_udpRecord")
    recordDataTable = recordData.find_element_by_tag_name("tbody")
    recordDataResult = recordDataTable.find_elements_by_tag_name("td")
    for data in recordDataResult:
        print (data.text, end = ', ')
    print ("")

In [ ]:
def ConnectToDBItReturnsConnector(hostDB, userId, userPw, targetDB, ):
    return pymysql.connect(host = hostDB, user = userId, password = userPw, db = targetDB, charset = "utf8")

In [ ]:
def GetDBCursor(connectorOfDB):
    return connectorOfDB.cursor()

In [ ]:
def ExecuteQuery(cursorOfDB, query):
    cursorOfDB.execute(query)

In [ ]:
def InsertDataToDB(query):
    connection = pymysql.connect(host = "lamb.kangnam.ac.kr", user = "stories2", password = "toortoor%^%", db = "stories2", charset = "utf8")
    print(query)
    #ExecuteQuery(GetDBCursor(connection), query)
    connection.commit()
    connection.close()
    #"insert into PitcherVSHitter value('test', 'test2', 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 1, 1);"

In [ ]:
def GetPlayerRecordData(webCrawl):
    recordData = webCrawl.find_element_by_id("cphContainer_cphContents_udpRecord")
    recordDataTable = recordData.find_element_by_tag_name("tbody")
    recordDataResult = recordDataTable.find_elements_by_tag_name("td")
    recordDataString = ""
    cnt = 0
    for data in recordDataResult:
        if cnt == 0:
            recordDataString = data.text
        else:
            recordDataString = recordDataString + ", " + data.text
        cnt += 1
    if cnt == 1:
        return "fail"
    else:
        return recordDataString

In [ ]:
#kbo 기록실 -> 투수 vs 타자 
ConnectToTargetUrl("http://www.koreabaseball.com/Record/Etc/HitVsPit.aspx")
pitcherTeamListLength = len(GetPitcherTeamList(webCrawl))
print (pitcherTeamListLength)

In [ ]:
print (GetPlayerRecordData(webCrawl))
if GetPlayerRecordData(webCrawl) == "fail":
    print ("ok")

In [ ]:
def GetBaseballData(webCrawl, pitcherTeamStartPoint, pitcherNameStartPoint, hitterTeamStartPoint, hitterNameStartPoint):
    #투수: 두산 ~ kt
    nowPitcherName = ""
    nowHitterName = ""
    nowPitcherTeam = ""
    nowHitterTeam = ""
    pitcherTeamListLength = len(GetPitcherTeamList(webCrawl))
    for pitcherTeamIndex in range(pitcherTeamStartPoint, pitcherTeamListLength):
        try:
            pitcherTeamObj = GetPitcherTeamList(webCrawl)[pitcherTeamIndex]
            nowPitcherTeam = pitcherTeamObj.text
            print ("-----pitcherTeam: " + nowPitcherTeam + "#" + str(pitcherTeamIndex) + " / " + str(pitcherTeamListLength) +"-----")
            pitcherTeamObj.click()
            
            pitcherNameListLength = len(GetPitcherNameList(webCrawl))
            #각 팀에 속해있는 선수들 명단 만큼
            for pitcherNameIndex in range(pitcherNameStartPoint, pitcherNameListLength):
                try :            
                    pitcherNameObj = GetPitcherNameList(webCrawl)[pitcherNameIndex]
                    pitcherNameObj.click()
                    nowPitcherName = pitcherNameObj.text
#                    print ("pitcher: " + nowPitcherName + " #" + str(pitcherNameIndex) + " / " + str(pitcherNameListLength))
                    
                    hitterTeamListLength = len(GetHitterTeamList(webCrawl))
                    #타자: 두산 ~ kt
                    for hitterTeamIndex in range(hitterTeamStartPoint, hitterTeamListLength):
                        try:
                            hitterTeamObj = GetHitterTeamList(webCrawl)[hitterTeamIndex]#
                            nowHitterTeam = hitterTeamObj.text
                            print ("-----hitterTeam: " + nowHitterTeam + "#" + str(hitterTeamIndex) + " / " + str(hitterTeamListLength) +"-----")
                            hitterTeamObj.click()
                            
                            hitterNameListLength = len(GetHitterNameList(webCrawl))
                            #각 팀에 속해있는 선수들 명단 만큼
                            for hitterNameIndex in range(hitterNameStartPoint, hitterNameListLength):
                                try :
                                    hitterNameObj = GetHitterNameList(webCrawl)[hitterNameIndex]
                                    hitterNameObj.click()
                                    nowHitterName = hitterNameObj.text
#                                    print ("hitter: " + nowHitterName + " #" + str(hitterNameIndex) + " / " + str(hitterNameListLength))
                                    print (nowPitcherName + "(#" + str(pitcherNameIndex) + ") vs " + nowHitterName + "(#" + str(hitterNameIndex) + ")")
#                                    
                                    try:
                                        btnSearchResult = GetSearchButton(webCrawl)
                                        btnSearchResult.click()
                                        time.sleep(0.4)
#                                        GetBaseBallDataTable(webCrawl)
#                                        ConnectToTargetUrl("http://www.koreabaseball.com/Record/Etc/HitVsPit.aspx")
                                        recordData = GetPlayerRecordData(webCrawl)
                                        #print (recordData)
                                        sqlQuery = "insert into PitcherVSHitter value('" + nowPitcherName +"', '" + nowHitterName + "',"
                                        if recordData != "fail":
                                            sqlQuery = sqlQuery + recordData + ", 1, 0, '" + nowPitcherTeam + "', '" + nowHitterTeam + "');"
                                            #print (sqlQuery)
                                        else:
                                            sqlQuery = sqlQuery + "0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0, 1, '" + nowPitcherTeam + "', '" + nowHitterTeam + "');"
                                        #print("SendQuery", sqlQuery)
                                        InsertDataToDB(sqlQuery)
                                    except:
                                        print ("except search endPoint: ", str(pitcherTeamIndex), str(pitcherNameIndex), str(hitterTeamIndex), str(hitterNameIndex))
                                except:
                                    print ("except hitter player")
                        except:
                            print ("except hitter team")
                except:
                    print ("except pitcher player")
        except:
            print ("except pitcher team")

In [ ]:
GetBaseballData(webCrawl, 1, 1, 1, 1)

In [ ]:

#for pitcherTeamIndex in range(1, len(pitcherOptions)):
#    print pitcherOptions[pitcherTeamIndex].text#get_attribute("value")
#    
#    #검색할 투수 팀을 고른다
#    pitcherOptions[pitcherTeamIndex].click()
#
#    time.sleep(random.randrange(1, 3))
#    #검색한 투수 팀에 속해있는 투수들의 리스트를 불러온다
#    pitcherNameList = webCrawl.find_element_by_id("cphContainer_cphContents_ddlPitcherPlayer")
#    pitcherNameListOptions = [data for data in pitcherNameList.find_elements_by_tag_name("option")]
#    
#    for pitcherNameIndex in range(1, len(pitcherNameListOptions)) :
#        try:
#            print pitcherNameListOptions[pitcherNameIndex].text#get_attribute("value")
#        except:
#            print "pitcherNameList #" + str(pitcherNameIndex) + " has Error"
        
    
    #for hitterTeamIndex in range(1, 2):
    #    print hitterOptions[hitterTeamIndex].get_attribute("value")
#for ele in pitcherOptions:
#    print ele.get_attribute("value")

In [ ]:
#cphContainer_cphContents_btnSearch